**Table of contents**<a id='toc0_'></a>    
- [ロボットアーム](#toc1_)    
- [pybulletの起動](#toc2_)    
- [初期設定](#toc3_)    
- [関数の定義](#toc4_)    
- [シミュレーションの実行](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[ロボットアーム](#toc0_)

本notebookでは「2軸のロボットアーム」と「オブジェクト」間の衝突判定を行う方法を紹介します。

（pybulletで使用可能な関数がまとめられたマニュアルについては[こちら](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf)を参照してください。）

# <a id='toc2_'></a>[pybulletの起動](#toc0_)

In [1]:
import pybullet
import pybullet_data
physics_client = pybullet.connect(pybullet.GUI) 

pybullet build time: Nov 28 2023 23:45:17


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


# <a id='toc3_'></a>[初期設定](#toc0_)
床の生成、ボックスオブジェクトの生成、ロボットの生成、カメラ位置の設定などの初期設定を行います。

In [2]:
pybullet.resetSimulation() # シミュレーション空間をリセット
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # pybulletに必要なデータへのパスを追加
pybullet.setGravity(0.0, 0.0, -9.8) # 地球上における重力に設定
time_step = 1./240.
pybullet.setTimeStep(time_step) # 1stepあたりに経過する時間の設定

#床の読み込み
plane_id = pybullet.loadURDF("plane.urdf")

# GUIモードの際のカメラの位置などを設定
camera_distance = 2.0
camera_yaw = 0.0 # deg
camera_pitch = -20 # deg
camera_target_position = [0.0, 0.0, 0.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

# ロボットの読み込み
arm_start_pos = [0, 0, 0.1]  # 初期位置(x,y,z)を設定
arm_start_orientation = pybullet.getQuaternionFromEuler([0,0,0])  # 初期姿勢(roll, pitch, yaw)を設定
arm_id = pybullet.loadURDF("../urdf/simple2d_arm.urdf",arm_start_pos, arm_start_orientation, useFixedBase=True) # ロボットが倒れないように、useFixedBase=Trueでルートのリンクを固定

# GUIモードの際のカメラの位置などを設定
camera_distance = 1.5
camera_yaw = 180.0 # deg
camera_pitch = -10 # deg
camera_target_position = [0.0, 0.0, 1.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

# ボックスの読み込み
## ボックスの重さ、サイズ、位置·姿勢を決める
mass = 5 # kg
box_size = [0.3, 0.6, 0.5]
position = [0.8, 0, 0.5]
orientation = [1, 0, 0, 0] # 四元数
box_collision_id = pybullet.createCollisionShape(pybullet.GEOM_BOX, halfExtents=box_size, physicsClientId=physics_client)
box_visual_id = pybullet.createVisualShape(pybullet.GEOM_BOX, halfExtents=box_size, physicsClientId=physics_client, rgbaColor=[1,0,0,1]) # 赤・半透明
box_body_id = pybullet.createMultiBody(mass, box_collision_id, box_visual_id, position, orientation, physicsClientId=physics_client)

ven = Mesa
ven = Mesa
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: target_position_vertual_link


# <a id='toc4_'></a>[関数の定義](#toc0_)
物体間の衝突の有効化/無効化に必要な、以下の関数を定義します。
- `GetNumLinks`：指定したオブジェクトのリンク数を取得
- `EnableCollisionBetweenObjects`：2つのオブジェクト間の衝突を有効/無効化する
- `JudgeContact`：2つのオブジェクトが接触しているかどうかを判定

In [3]:
def get_num_links(objId):
    """
    指定されたオブジェクトのリンク数を取得する。
    Parameters
    ----------
    objId : int
        オブジェクトのインデックス。
        
    Returns
    -------
    linkNum : int
        オブジェクトのリンク数
    """
    current_link_idx = 0
    link_num = 0
    while True:
        result = pybullet.getLinkState(objId, current_link_idx)
        if result is None:
            link_num = current_link_idx + 1
            break
        current_link_idx += 1
    return link_num

def enable_collision_between_objects(obj1_id, obj2_id, enable):
    """
    指定されたオブジェクト間の衝突を有効/無効にする。
    Parameters
    ----------
    obj1_id : int
        1つ目のオブジェクトのインデックス。
    obj2_id : int
        2つ目のオブジェクトのインデックス。
    enable : bool
        衝突判定を有効にする場合はTrue、無効にする場合はFalse。

    Returns
    -------
    None
    """
    # 各オブジェクトのリンク数を取得
    obj1_link_num = get_num_links(obj1_id)
    obj2_link_num = get_num_links(obj2_id)

    # 衝突判定の有効/無効を設定
    for obj1_link_idx in range(-1, obj1_link_num):
        for obj2_link_idx in range(-1, obj2_link_num):
            pybullet.setCollisionFilterPair(obj1_id, obj2_id, obj1_link_idx, obj2_link_idx, enable)

def judge_collision(obj1_id, obj2_id):
    """
    2つのオブジェクトが接触しているかどうかを判定する。
    Parameters
    ----------
    obj1_id : int
        1つ目のオブジェクトのインデックス。
    obj2_id : int
        2つ目のオブジェクトのインデックス。

    Returns
    -------
    isCollision : bool
        2つのオブジェクトが接触している場合はTrue、接触していない場合はFalse。
    """
    # 2つのオブジェクト間の接触情報を取得
    pts = pybullet.getClosestPoints(obj1_id, obj2_id, distance=100)

    # 2つのオブジェクトが接触しているかどうかを判定
    is_collision = False
    for pt in pts:
        distance = pt[8]
        # 2つのオブジェクト間で接触している点（=距離が0未満の点）があるかを判定
        if distance < 0.0:
            is_collision = True
            break
	
    return is_collision

# <a id='toc5_'></a>[シミュレーションの実行](#toc0_)
ロボットアームを動かし、オブジェクトとの衝突判定を行います。

下記コードを実行すると、GUI画面上のロボットアームをマウスで操作することができ、ロボットアームをボックスに触れさせると、画面上に「collision」と表示されます。

In [4]:
import time

pybullet.setRealTimeSimulation(1) # リアルタイムシミュレーションを有効化（GUI画面上で、オブジェクトをマウスで操作することが可能になる）

# ロボットアームとボックス間の衝突判定を無効化
enable_collision_between_objects(arm_id, box_body_id, False)

# ロボットアームとボックスが衝突しているかどうかを画面上に表示
while (pybullet.isConnected()):

	# ロボットアームとボックスが衝突しているかどうかを判定
	is_collision = judge_collision(arm_id, box_body_id)

	# 衝突している場合は画面上に表示
	if is_collision:
		pybullet.addUserDebugText("collision", [0.5, 0.0, 1.5], textSize=5, lifeTime=0.1, textColorRGB=[1,0,0])
	
	time.sleep(time_step)